In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/act-pas/active_passive_full_cleaned.tsv
/kaggle/input/act-pas/active_passive.csv


In [3]:
import pandas as pd

In [4]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
df = pd.read_csv("/kaggle/input/act-pas/active_passive_full_cleaned.tsv", sep="\t", header=None)

df.columns = ["Passive", "Active"]

In [6]:
df.head()

,Passive,Active
0,In cases of dispute the matter shall be resolv...,In cases of dispute the judgement of the twent...
1,A dispute on this point shall be determined in...,The judgement of equals shall determine in the...
2,This matter shall be resolved by the judgement...,The judgement of his equals shall resolve this...
3,"If any Freeman shall die Intestate, his chatte...","If any Freeman shall die Intestate, the hands ..."
4,All evil customs connected with forests and Wa...,Twelve sworn Knights of the same County chosen...


In [8]:
!pip uninstall -y transformers
!pip install transformers
!pip install accelerate -U

  Using cached transformers-4.39.3-py3-none-any.whl.metadata (134 kB)
Using cached transformers-4.39.3-py3-none-any.whl (8.8 MB)


In [9]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model_checkpoint = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

2024-04-10 18:42:41.028030: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 18:42:41.028168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 18:42:41.121263: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-pas2act",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [11]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00a 0:00:01


In [12]:
from evaluate import load
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a8df2d097266b23c34a6b066d1e27218b331c618e7a8007154e3d9fc299975c8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [13]:
metric1 = load("rouge")

In [14]:
metric2 = load("bleu")

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [16]:
df2 = pd.read_csv("/kaggle/input/act-pas/active_passive.csv")


In [17]:
df2.head()

,Active,Passive
0,He reads a novel.,A novel is read.
1,He does not cook food.,Food is not cooked by him.
2,Does he purchase books?,Are books being purchased by him?
3,They grow plants.,Plants are grown by them.
4,She teaches me.,I am taught by her.


In [40]:
df1 = pd.concat([df2, df], ignore_index=True)

In [41]:
df1

,Active,Passive
0,He reads a novel.,A novel is read.
1,He does not cook food.,Food is not cooked by him.
2,Does he purchase books?,Are books being purchased by him?
3,They grow plants.,Plants are grown by them.
4,She teaches me.,I am taught by her.
...,...,...
472670,"An adopted daughter, whom she had rescued from...",She was led into the church by an adopted daug...
472671,The country well should pay such a servant of ...,Such a servant of the country should be well p...
472672,"Governor Andrew, of Massachusetts sent to me f...",Harriet was sent to me from Boston by Governor...
472673,"General Hunter employed her, and I think by ge...","She was employed by General Hunter, and I thin..."


In [42]:
df1['Passive_word_count'] = df1['Passive'].apply(lambda x: len(x.split()))

# Filter out rows where the number of words in "Active" is more than 20
df_filtered = df1[df1['Passive_word_count'] <= 16]

# Drop the temporary column 'Active_word_count'
df1 = df_filtered.drop(columns=['Passive_word_count'])

In [43]:
df1

,Active,Passive
0,He reads a novel.,A novel is read.
1,He does not cook food.,Food is not cooked by him.
2,Does he purchase books?,Are books being purchased by him?
3,They grow plants.,Plants are grown by them.
4,She teaches me.,I am taught by her.
...,...,...
472662,"But to - day, our senatorial charlatans and th...","But to-day, what is politically proposed by ou..."
472667,A roar of satisfaction from the crowd below re...,This was responded to by a roar of satisfactio...
472669,Various names among her Southern friends knew ...,Harriet was known by various names among her S...
472671,The country well should pay such a servant of ...,Such a servant of the country should be well p...


In [44]:
max_input_length = 128
max_target_length = 128

def preprocess_function(row):
    inputs = row["Passive"] + "</s>"
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=row["Active"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [45]:
tokenized__data = df1.apply(preprocess_function, axis=1)


In [46]:
tokenized__data

0         [input_ids, attention_mask, labels]
1         [input_ids, attention_mask, labels]
2         [input_ids, attention_mask, labels]
3         [input_ids, attention_mask, labels]
4         [input_ids, attention_mask, labels]
                         ...                 
472662    [input_ids, attention_mask, labels]
472667    [input_ids, attention_mask, labels]
472669    [input_ids, attention_mask, labels]
472671    [input_ids, attention_mask, labels]
472674    [input_ids, attention_mask, labels]
Length: 157481, dtype: object

In [47]:
from transformers import AutoModelForSeq2SeqLM


In [48]:
from sklearn.model_selection import train_test_split

# Step 1: Split the data into train and combined validation/test sets
train_data, val_test_data = train_test_split(tokenized__data, test_size=0.2, random_state=42)

# Step 2: Further split the combined validation/test set into separate validation and test sets
val_data, test_data = train_test_split(val_test_data, test_size=0.5, random_state=42)

# Now you have train_data, val_data, and test_data


In [49]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [50]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric1.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [51]:
train_dataset = train_data.to_frame().to_dict(orient='records')
val_dataset = val_data.to_frame().to_dict(orient='records')
test_dataset=test_data.to_frame().to_dict(orient='records')

In [52]:
train_dataset1 = [v for d in train_dataset for k, v in d.items()]
val_dataset1=[v for d in val_dataset for k, v in d.items()]

In [53]:
print(train_dataset1[:10])

[{'input_ids': [37, 16696, 21, 3, 13505, 135, 47, 8744, 91, 57, 2788, 18, 553, 35, 288, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [2788, 3, 18, 7281, 288, 8744, 91, 8, 16696, 21, 3, 13505, 135, 5, 1]}, {'input_ids': [15364, 18154, 30, 8, 294, 13, 224, 15484, 7, 47, 92, 4344, 3, 60, 12760, 57, 388, 76, 5451, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [1140, 76, 5451, 92, 4344, 3, 60, 12760, 15364, 18154, 30, 8, 294, 13, 224, 15484, 7, 5, 1]}, {'input_ids': [2351, 15937, 13, 8, 554, 28005, 1246, 16, 10438, 19, 5293, 15, 26, 57, 8, 4708, 12042, 7, 44, 8067, 440, 5, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [37, 4708, 12042, 7, 44, 8067, 440, 5293, 430, 15937, 13, 8, 554, 28005, 1246, 16, 10438, 5, 1]}, {'input_ids': [3, 21900, 8, 740, 47, 5136, 57, 135, 11, 3353, 7620, 2726, 423, 5, 1, 1], 'attention_mask': [1

In [54]:
from huggingface_hub import notebook_login

notebook_login()

In [55]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset1 ,
    eval_dataset=val_dataset1,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [56]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.220300,0.161110,97.659200,93.929000,95.252600,95.275400,14.936700
2,0.176200,0.135157,97.898100,94.701600,95.717700,95.739100,14.946900
3,0.166000,0.126286,97.967900,94.904000,95.839400,95.857800,14.944800
4,0.150900,0.123196,97.981200,94.954600,95.871400,95.897300,14.946400


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=15748, training_loss=0.20539188385009766, metrics={'train_runtime': 3557.7615, 'train_samples_per_second': 141.644, 'train_steps_per_second': 4.426, 'total_flos': 3910143068602368.0, 'train_loss': 0.20539188385009766, 'epoch': 4.0})

In [58]:
print("hello")

hello


In [59]:
test_dataset1=[v for d in test_dataset for k, v in d.items()]


In [60]:
filtered_labels = [label for label in test_dataset1[0]['labels'] if label >= 0]

# Decode the filtered labels
decoded_labels = tokenizer.decode(filtered_labels, skip_special_tokens=True)
print(decoded_labels)


The barking of Cannon and the fluttering of sails answered it.


In [64]:

test_predictions = trainer.predict(test_dataset1[:10])
t=test_predictions.predictions
test_predictions

PredictionOutput(predictions=array([[    0,    37, 21696,    53,    13,   205, 17805,    11,     8,
            3,    89, 20224,    53,    13, 14725,     7,  9986,    34,
            5,     1],
       [    0,    37,   905,  6891, 19170, 11664,   107,     6,     3,
           29, 27627,   160,     3,     9,  8674,    12,     8,   388,
            5,     1],
       [    0,  6783,    12,   507,  3628,     8, 14751,  7889,    13,
            8,  9964,   141, 18277,   662,   648,    34,     5,     1,
            0,     0],
       [    0,    71,    89, 22637,  3211, 10933,   120,   164, 26395,
          237,  1322, 11811,     5,     1,     0,     0,     0,     0,
            0,     0],
       [    0,   466,    84,     3,     9, 13106,   376,    44,     8,
          828,    59,   659,  4632,   112, 16370,     5,     1,     0,
            0,     0],
       [    0, 26805,  1553,    48, 14193,    30,  1619,   239,     6,
          898,  3166,     5,     1,     0,     0,     0,     0,     0,
    

In [65]:
predicted_sentences = []
for prediction in test_predictions.predictions:
    tokens = tokenizer.decode(prediction, skip_special_tokens=True)
    predicted_sentences.append(tokens)



# Print the predicted sentences

for i, sentence in enumerate(predicted_sentences):
    filtered_labels = [label for label in test_dataset1[i]['labels'] if label >= 0]

# Decode the filtered labels
    decoded_labels = tokenizer.decode(filtered_labels, skip_special_tokens=True)

    print(f"Example {i+1} - {decoded_labels} Predicted Sentence: {sentence}")

Example 1 - The barking of Cannon and the fluttering of sails answered it. Predicted Sentence: The barking of Cannon and the fluttering of sails answered it.
Example 2 - The account greatly shocked Edith, notwithstanding her aversion to the man. Predicted Sentence: The account greatly shocked Edith, notwithstanding her aversion to the man.
Example 3 - Prior to 1844 the legislative assembly of the territory had investigated four times it. Predicted Sentence: Prior to 1844 the legislative assembly of the territory had investigated four times it.
Example 4 - Aerial attack enormously may hamper even land transit. Predicted Sentence: Afghan attack enormously may hamper even land transit.
Example 5 - That which awaited him at the office not lightened his spirits. Predicted Sentence: That which awaited him at the office not lightened his spirits.
Example 6 - Milton began this poem on Christmas day, 1629. Predicted Sentence: Milton began this poem on Christmas day, 1629.
Example 7 - After a wh

In [68]:
input_text = "My work is done by him"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
print(input_ids)

# Convert input_ids tensor to a dictionary
input_features = {"input_ids": input_ids[0]}

# Generate predictions
output = trainer.predict([input_features])
output
tokenizer.decode(output.predictions[0], skip_special_tokens=True)
# input_features

tensor([[499, 161,  19, 612,  57, 376,   1]])


'He does my work'

In [69]:
trainer.push_to_hub("Pushparaj20/passive")

events.out.tfevents.1712775325.56e49bb9f7b9.34.0:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Pushparaj20/t5-small-pas2act/commit/e0607a7a6e3c004671b674806b9003018b929dcb', commit_message='Pushparaj20/passive', commit_description='', oid='e0607a7a6e3c004671b674806b9003018b929dcb', pr_url=None, pr_revision=None, pr_num=None)

In [76]:
from transformers import AutoModelForSeq2SeqLM

# Load the model from the model hu
model = AutoModelForSeq2SeqLM.from_pretrained("Pushparaj20/t5-small-pas2act")

# Now you can use the loaded model for inference or fine-tuning
input_text = "My work was done by him"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Generate predictions
outpu = model.generate(input_ids)
outpu

tensor([[  0, 216, 410,  82, 161,   1]])

In [77]:
b=outpu.numpy()
tokenizer.decode(b[0], skip_special_tokens=True)

'He did my work'